In [ ]:
from subprocess import Popen
from subprocess import PIPE
from scipy.signal.windows import gaussian
import matplotlib
import random
from matplotlib import pyplot as plt
import subprocess
from datetime import datetime
import time
from time import sleep
import re
import numpy as np
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [8, 8]
from datetime import datetime
import cv2
import heapq
import glob
from scipy.signal import convolve, convolve2d
from threading import Thread

In [ ]:
THRESHOLD = 0.5
PRINT_THRESHOLD = 0.5
TAP_SHAKE_RAD = 5

def now():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
def tap(x,y):
    x += np.random.randint(-TAP_SHAKE_RAD, TAP_SHAKE_RAD + 1)
    y += np.random.randint(-TAP_SHAKE_RAD, TAP_SHAKE_RAD + 1)
    assert(isinstance(x, int) and isinstance(y, int))
    proc = subprocess.Popen(args = "adb shell input tap {} {}".format(x,y), shell = True)
    
def swipe(x0,y0,x1,y1,dur = 100):
#     assert(isinstance(x, int) and isinstance(y, int))
    proc = subprocess.Popen(args = "adb shell input swipe {} {} {} {} {}".format(x0,y0,x1,y1,dur), shell = True)
    
def read_image(filename):
    return cv2.imread(filename, cv2.IMREAD_UNCHANGED)

def raw_screenshot():
    proc = Popen(args = "adb shell screencap -p", shell = True, stdout = PIPE)
    raw = ""
    while proc.poll() is None:
        stdout, _= proc.communicate()
        raw += stdout
    raw = raw.replace("\r\n", "\n")
    return raw
    

def take_screenshot():
    raw = raw_screenshot()
    im = cv2.imdecode(np.frombuffer(raw, np.uint8), cv2.IMREAD_UNCHANGED)
    return im
    
def save_screenshot(filename = None):
    filename = filename or ("screencap" + datetime.now().strftime("%Y%m%dT%H%M%S") + ".png")
    proc = subprocess.Popen(args = "adb shell screencap -p > " + filename, shell = True)
    return proc

screen = take_screenshot()
SCREEN_WIDTH = screen.shape[1]
SCREEN_HEIGHT = screen.shape[0]


class Image(object):
    def __init__(self, name, image):
        self.name = name
        self.im = image
        self.shape = image.shape
        self._canny = None
        self._scale_canny = {}
        self._gray = None
        self._scale_gray = {}
        
    def canny(self):
        if self._canny is None:
            self._canny = cv2.Canny(self.im, 100, 200)
        return self._canny
    
    def gray(self):
        if self._gray is None:
            self._gray = cv2.cvtColor(self.im, cv2.COLOR_BGR2GRAY)
        return self._gray
        
    def show(self):
        fig = plt.figure()
        fig.set_figheight(16)
        fig.set_figwidth(9)
        if len(self.im.shape) >= 3:
            image = self.im.copy()
            image[:,:,(2,1,0)] = image[:,:,(0,1,2)]
            plt.imshow(image, aspect= 'equal')
        else:
            plt.imshow(image, cmap = "gray", aspect = 'equal')
        plt.show()
        
    def scale_canny(self,factor):
        if factor in self._scale_canny:
            return self._scale_canny[factor]
        if factor == 1:
            return self.canny()
        elif factor > 1:
            im = cv2.resize(self.im, None, 0, fx = factor, fy = factor)
            im = cv2.Canny(im, 100, 200)
            self._scale_canny[factor] = im
            return im
        else:
            im = cv2.resize(self.im, None, 0, fx = factor, fy = factor, interpolation = cv2.INTER_AREA)
            im = cv2.Canny(im, 100, 200)
            self._scale_canny[factor] = im
            return im
        
    def scale_gray(self,factor):
        if factor in self._scale_gray:
            return self._scale_gray[factor]
        if factor == 1:
            return self.gray()
        elif factor > 1:
            im = cv2.resize(self.im, None, 0, fx = factor, fy = factor)
        else:
            im = cv2.resize(self.im, None, 0, fx = factor, fy = factor, interpolation = cv2.INTER_AREA)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        self._scale_gray[factor] = im
        return im

class Screenshot(Image):
    def __init__(self, name, image, t):
        super(Screenshot, self).__init__(name, image)
        self.time = t
    
class Template(Image):
    def __init__(self, name, image, tolerance = THRESHOLD, **kwargs):
        """
        template constructor, with optional arguments to bound where on the screen to search, and
        if there is a guess as to where to search.
        """
        super(Template, self).__init__(name, image)
        self.tolerance = tolerance
        if "guess" in kwargs:
            guess = kwargs["guess"]
            self.xmin = max(guess[0] - 2 * image.shape[1], 0)
            self.xmax = min(guess[0] + 2 * image.shape[1], SCREEN_WIDTH - 1)
            self.ymin = max(guess[1] - 2 * image.shape[0], 0)
            self.ymax = min(guess[1] + 2 * image.shape[0], SCREEN_HEIGHT - 1)
            
        else:
            self.xmin = kwargs.get("xmin", 0)
            self.xmax = kwargs.get("xmax", SCREEN_WIDTH - 1)
            self.ymin = kwargs.get("ymin", 0)
            self.ymax = kwargs.get("ymax", SCREEN_HEIGHT - 1)
            
#load in the assets
            
DIGITS = {i: Template("{:d}".format(i), read_image("./digits/{:d}.png".format(i))) for i in range(10)}
TENT = Template("tent", read_image("./tent.png"))
TREE = Template("tree", read_image("./tree.png"))
NEXT_PUZZLE_BUTTON = Template("play button", read_image("./next_puzzle.png"))
OK_BUTTON = Template("ok button", read_image("./ok.png"))

    
def show_image(image):
    fig = plt.figure()
    fig.set_figheight(16)
    fig.set_figwidth(9)
    if len(image.shape) == 3:
        image = image.copy()
        image[:,:,(2,1,0)] = image[:,:,(0,1,2)]
        plt.imshow(image, aspect= 'equal')
    else:
        plt.imshow(image, cmap = "gray", aspect = 'equal')
    plt.show()

def find_template_all(image, template, threshold = THRESHOLD):
    if len(template.shape) == 3 and template.shape[2]==4:
        mask = template[:,:,3]
        mask = np.stack([mask] * template.shape[2], axis = -1)
    else:
        mask = np.ones(template.shape)
    res = cv2.matchTemplate(image, template, method = cv2.TM_CCORR_NORMED, mask = template_mask)
    found = []
    res[res < threshold] = 0
    while np.any(res):
        loc = tuple(np.argwhere(res == res.max())[0])
        found.append(loc)
        x_min = max(loc[0] - template.shape[0]/2, 0)
        x_max = min(loc[0] + template.shape[0]/2, res.shape[0]-1)
        y_min = max(loc[1] - template.shape[1]/2, 0)
        y_max = min(loc[1] + template.shape[1]/2, res.shape[1]-1)
        res[x_min:x_max,y_min:y_max] = 0
    return found

def get_max_loc(ar):
    return np.unravel_index(ar.argmax(), ar.shape)

def find_template(image, template):
    res = cv2.matchTemplate(image, template, method = cv2.TM_CCORR_NORMED)
    y,x = get_max_loc(res)
    return x,y,np.max(res)

def search(image, template):
    height, width = image.shape[:2]
    im = image.im[template.ymin : template.ymax, template.xmin : template.xmax]
    x, y, confidence = search_ar(im, template.im)
    x = x + template.xmin
    y = y + template.ymin
    return x + template.shape[1]/2,y + template.shape[0]/2, confidence

def search_canny(image, template):
    height, width = image.shape[:2]
    im = image.canny()[template.ymin : template.ymax, template.xmin : template.xmax]
    x, y, confidence = search_ar(im, template.canny())
    x = x + template.xmin
    y = y + template.ymin
    return x + template.shape[1]/2,y + template.shape[0]/2, confidence

def search_ar(image, template):
    similarity = cv2.matchTemplate(image, template, method = cv2.TM_CCORR_NORMED)
    y,x = get_max_loc(similarity)
    confidence = similarity.max()
    return x, y, confidence

def match_canny(image, template, threshold = THRESHOLD):
    """searches the image for template using edges and returns a list of points and bounding boxes"""
    height, width = template.shape[:2]
    similarity = cv2.matchTemplate(image.canny(), template.canny(), method = cv2.TM_CCORR_NORMED)
    similarity[similarity < threshold] = 0
    locations = []
    while np.any(similarity):
        y,x = get_max_loc(similarity)
        locations.append((x,y))
        similarity[y - height / 2 : y + height / 2, x - width / 2 : x + width/2] = 0 #non-maximum suppresion
    return locations

def match_canny_scale(image, template, threshold = THRESHOLD, scale = 1.0):
    """searches the image for template using edges and returns a list of points and bounding boxes"""
    height, width = template.scale_canny(scale).shape[:2]
    similarity = cv2.matchTemplate(image.canny(), template.scale_canny(scale), method = cv2.TM_CCORR_NORMED)
    similarity[similarity < threshold] = 0
    locations = []
    while np.any(similarity):
        y,x = get_max_loc(similarity)
        locations.append((x + width / 2 ,y + height / 2))
        similarity[y - height / 2 : y + height / 2, x - width / 2 : x + width/2] = 0 #non-maximum suppresion
    return locations

def match_gray_scale(image, template, threshold = THRESHOLD, scale = 1.0):
    height, width = template.scale_gray(scale).shape[:2]
    gray = image.gray()
    similarity = cv2.matchTemplate(np.invert(gray),
                                   np.invert(template.scale_gray(scale)),
                                   method = cv2.TM_CCORR_NORMED)
    similarity[similarity < threshold] = 0
    locations = []
    plt.show()
    while np.any(similarity):
        y,x = get_max_loc(similarity)
        locations.append((x + width / 2 ,y + height / 2))
        similarity[y - height / 2 : y + height / 2, x - width / 2 : x + width/2] = 0 #non-maximum suppresion
    return locations    
    


def find_template_scale(image, template, scales = None):
    if scales is None:
        scales = np.linspace(0.8,4,301)
    best_scale = None
    max_similarity = 0
    for factor in scales:
        similarity = cv2.matchTemplate(image.canny(), template.scale_canny(factor), method = cv2.TM_CCORR_NORMED)
        if similarity.max() > max_similarity:
            max_similarity = similarity.max()
            best_scale = factor
    return best_scale, max_similarity

def match_canny_scales(image, template, threshold = THRESHOLD, scales = None):
    scale, confidence = find_template_scale(image, template, scales)
    return match_canny_scale(image, template, threshold, scale), scale, confidence

def which_digit(image, threshold = THRESHOLD, scale = 1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    best_digit = -1
    confidence = -1
    for i in range(10):
        score = cv2.matchTemplate(np.invert(gray),
                                  np.invert(DIGITS[i].scale_gray(scale)),
                                  method = cv2.TM_CCORR_NORMED).max()
        if score > confidence:
            best_digit = i
            confidence = score
    return best_digit, confidence
        
    
def non_maximum_suppression(signal, threshold = 0.6, peak_radius = 40):
    peaks = []
    signal = signal.copy()
    m = signal.max()
    signal[signal < threshold * m] = 0
    while signal.max() > 0:
        maxi = np.argmax(signal)
        bin_left = max(maxi - peak_radius,0)
        bin_right = min(maxi + peak_radius, signal.size)
        peak = signal[bin_left: bin_right]
        peak /= peak.sum() # normalize to compute weighted mean of peak
        peak = int((peak * np.arange(peak.size)).sum()) # normalized mean
        peaks.append(peak + bin_left)  
        signal[bin_left: bin_right] = 0
    return np.array(sorted(peaks))


def combine_similar(l, radius = 40):
    """assumes integer points"""
    l = sorted(l)
    processed = -1
    new_list = []
    for point in l:
        if point <= processed:
            continue
        same_point = [point]
        processed = point
        for point2 in l:
            if point2 <= point:
                continue
            if point2 <= point + radius:
                same_point.append(point2)
                processed = point2
            else:
                break
        new_list.append(int(np.mean(same_point)))
    return new_list
            

In [ ]:
# from sklearn.linear_model import LinearRegression LR
class GridModel(object):
    def fit(self, hor_peaks, vert_peaks, tolerance = 0.1, symmetric = True):
        self.tolerance = tolerance
        hor_peaks = np.array(hor_peaks).copy().astype(np.float).reshape(-1)
        vert_peaks = np.array(vert_peaks).copy().astype(np.float).reshape(-1)
        hor_sep, vert_sep, avg_sep = self.calculate_separation(hor_peaks, vert_peaks)
        if hor_sep.size == 0 or vert_sep.size == 0:
            self.success = False
            return False
        count = 0
        while (np.any(np.abs((hor_sep - avg_sep) / avg_sep) > tolerance) or
               np.any(np.abs((vert_sep - avg_sep) / avg_sep) > tolerance) or
               (symmetric and (len(hor_peaks) != len(vert_peaks) or 6 > len(hor_peaks) or len(hor_peaks) > 19))):

            count += 1
            if count > 1000:
#                 print "maximum iterations exceeded"
                break
            #check if missing peak
            if len(hor_peaks) < 23:
                index, gap = self.find_peaks_if_missing(hor_sep, avg_sep)
                if 5 > gap > 0:
                    hor_peaks = np.insert(hor_peaks, index + 1, [hor_peaks[index] + i * avg_sep for i in range(1,gap+1)])
                    hor_peaks = np.round(hor_peaks)
                    hor_sep, vert_sep, avg_sep = self.calculate_separation(hor_peaks, vert_peaks)
                    continue
            if len(vert_peaks) < 23:
                index, gap = self.find_peaks_if_missing(vert_sep, avg_sep)
                if 5 > gap > 0:
                    vert_peaks = np.insert(vert_peaks, index + 1, [vert_peaks[index] + i * avg_sep for i in range(1,gap+1)])
                    vert_peaks = np.round(vert_peaks)
                    hor_sep, vert_sep, avg_sep = self.calculate_separation(hor_peaks, vert_peaks)
                    continue
            #check if false peak
            if len(hor_peaks) > 6:
                index = self.find_false_peak(hor_sep, avg_sep)
                if index >= 0:
                    hor_peaks = np.delete(hor_peaks, index)
                    hor_sep, vert_sep, avg_sep = self.calculate_separation(hor_peaks, vert_peaks)
                    continue
            if len(vert_peaks) > 6:
                index = self.find_false_peak(vert_sep, avg_sep)
                if index >= 0:
                    vert_peaks = np.delete(vert_peaks, index)
                    hor_sep, vert_sep, avg_sep = self.calculate_separation(hor_peaks, vert_peaks)
                    continue
        else:          
        
            if len(hor_peaks) == len(vert_peaks) or not symmetric:
                self.hor_offset = min(hor_peaks)
                self.vert_offset = min(vert_peaks)
                self.sep = avg_sep
                self.n = len(hor_peaks)
                self.n_vert = len(vert_peaks)
                self.success = True
            else:
                self.success = False
            return max(np.max((np.abs((hor_sep - avg_sep) / avg_sep))),
                       np.max(np.abs((vert_sep - avg_sep) / avg_sep)))
        self.success = False
        return max(np.max((np.abs((hor_sep - avg_sep) / avg_sep))),
                   np.max(np.abs((vert_sep - avg_sep) / avg_sep)))
    
    def calculate_separation(self, hor_peaks, vert_peaks):
        hor_sep = hor_peaks[1:] - hor_peaks[:-1]
        vert_sep = vert_peaks[1:] - vert_peaks[:-1]
        trim = (len(hor_sep) + len(vert_sep))/4
        avg_sep = np.sort(np.concatenate((hor_sep,vert_sep)))[trim:-trim].mean()
        return hor_sep, vert_sep, avg_sep
    
    def find_peaks_if_missing(self, sep, avg):
        #returns index and number of peaks to insert
        gapi = np.argmax(sep)
        gap = round(sep[gapi] / avg)
        if gap <= 1:
            return 0,0
        if abs(sep.max() / avg / gap - 1) < self.tolerance:
            return gapi, int(gap) - 1
        else:
            return 0,0
        
    def find_false_peak(self, sep, avg):
        for gapi,gap in enumerate(sep):
            if abs(gap / avg - 1) > self.tolerance:
                if gapi == len(sep) - 1:
                    return gapi + 1
                if abs(sep[gapi+1] / avg - 1) > self.tolerance:
                    return gapi + 1
                else:
                    return gapi
        return -1
        
    def transform(self):
        return ((self.hor_offset + np.arange(self.n) * self.sep).astype(int),
                (self.vert_offset + np.arange(self.n) * self.sep).astype(int))
            
    def fit_transform(self, hor_peaks, vert_peaks, **kwargs):
        self.fit(hor_peaks, vert_peaks, **kwargs)
        if self.success:
            return self.transform()
        else:
            return False, False
    
def pretify_game_array(ar, top, left):
    d = {0:' ',
         1:u'T',
         2:u'\U0000028C',
         3:'_'}
    s = u" "
    s += u"".join(map(str, top)) + "\n"
    for i in range(len(left)):
        s += str(left[i])
        s += u"".join(map(lambda x: d[x], ar[i])) + "\n"
    return s
    

In [ ]:
ADJACENT_KERNEL = np.array([[0,1,0], [1,0,1], [0,1,0]])
DIAGONAL_KERNEL = np.array([[1,0,1], [0,0,0], [1,0,1]])

class State(object):
    def __init__(self, array, top, left):
        self.grid = np.array(array).copy()
        self.top = np.array(top).copy()
        self.left = np.array(left).copy()
        assert (len(self.left), len(self.top)) == self.grid.shape
        self.size = self.grid.shape[0]
        self.log = []
        
    def back(self):
        i,j,_ = self.log.pop()
        self.grid[i,j] = 0
    
    def set_blank(self, i, j):
        """sets to blank if unknown"""
        if self.grid[i,j] == 0:
            self.grid[i,j] = 3
            self.log.append((i,j,3))
    
    def set_tent(self, i, j):
        if self.grid[i,j] == 0:
            self.grid[i,j] = 2
            self.log.append((i,j,2))
            for key in self.get_adjacent(i,j):
                self.set_blank(*key)
            for key in self.get_diagonal(i,j):
                self.set_blank(*key)
            
    def get_adjacent(self, i, j):
        r = {}
        if i > 0:
            r[i - 1, j] = self.grid[i - 1, j]
        if i < self.size - 1:
            r[i + 1, j] = self.grid[i + 1, j]
        if j > 0:
            r[i, j - 1] = self.grid[i, j - 1]
        if j < self.size - 1:
            r[i, j + 1] = self.grid[i, j + 1]
        return r
    
    def get_diagonal(self, i, j):
        r = {}
        if i > 0 and j > 0:
            r[i - 1, j - 1] = self.grid[i - 1, j - 1]
        if i > 0 and j < self.size - 1:
            r[i - 1, j + 1] = self.grid[i - 1, j + 1]
        if i < self.size - 1 and j > 0:
            r[i + 1, j - 1] = self.grid[i + 1, j - 1]
        if i < self.size - 1 and j < self.size - 1:
            r[i + 1, j + 1] = self.grid[i + 1, j + 1]
        return r
            
    def is_correct(self):
        # puzzle is completed
        if np.any(self.grid == 0):
            return False
        
        #the rows numbers are correct
        if np.any(self.left != np.sum(self.grid == 2, axis = 1)):
            return False
        
        #the column numbers are correct
        if np.any(self.top != np.sum(self.grid == 2, axis = 0)):
            return False
        
        # no tents are adjacent
        tents = self.grid == 2
        if (np.any(tents[1:] * tents[:-1]) or
            np.any(tents[:,1:] * tents[:,:-1]) or
            np.any(tents[1:,1:] * tents[:-1,:-1]) or 
            np.any(tents[1:,:-1] * tents[:-1, 1:])):
            return False
        
        # every tree has a corresponding adjacent tent
        return self._tent_tree_dfs()        
        
    def _tent_tree_dfs(self, index = 0, paired_tents = None):
        i = index / self.size
        j = index % self.size
        if paired_tents is None:
            paired_tents = set()
        if self.grid[i,j] == 1:
            for key, value in self.get_adjacent(i,j).items():
                #this lookup has a branching factor of at most 2 (opposite sides of the tree)
                if value == 2 and key not in paired_tents:
                    paired_tents.add(key)
                    index += 1
                    if index == self.size * self.size:
                        return True
                    if self._tent_tree_dfs(index, paired_tents):
                        return True
                    paired_tents.remove(key)
        else:
            index += 1
            if index == self.size * self.size:
                return True
            else:
                return self._tent_tree_dfs(index, paired_tents)
            
    def _not_adjacent_to_tree(self):
        blanks = convolve2d(self.grid == 1, ADJACENT_KERNEL, mode = "same") < 1
        for i in range(self.size):
            for j in range(self.size):
                if blanks[i,j]:
                    self.set_blank(i,j)
        return self
                    
    def _row_col_deductions(self):
        unknowns = self.grid == 0
        tents = self.grid == 2
        row_unknowns = unknowns.sum(axis = 1)
        col_unknowns = unknowns.sum(axis = 0)
        row_tents = tents.sum(axis = 1)
        col_tents = tents.sum(axis = 0)
        for i in range(self.size):
            #row deductions
            if row_unknowns[i] + row_tents[i] == self.left[i]: # only tents left
                for j in range(self.size):
                    self.set_tent(i,j)
            if row_tents[i] == self.left[i]: # only blanks left
                for j in range(self.size):
                    self.set_blank(i,j)
            #col deductions
            if col_unknowns[i] + col_tents[i] == self.top[i]: # only tents left
                for j in range(self.size):
                    self.set_tent(j,i)
            if col_tents[i] == self.top[i]: # only blanks left
                for j in range(self.size):
                    self.set_blank(j,i)
        return self
    
    def _pairing_deductions(self):
        for i1 in range(self.size):
            for j1 in range(self.size):
                if self.grid[i1,j1] == 1:
                    neighbours = self.get_adjacent(i1, j1)
                    num_neighbours = (np.array(neighbours.values()) % 2 == 0).sum()
                    if num_neighbours == 1:
                        for key in neighbours:
                            self.set_tent(*key)
                    elif num_neighbours == 2:                    
                        for i2 in range(max(0, i1-2), min(self.size, i1+3)):
                            for j2 in range(max(0, j1-2), min(self.size, j1+3)):
                                if (i1,j1) == (i2,j2):
                                    continue
                                if self.grid[i2,j2] == 1:
                                    neighbours2 = {}
                                    neighbours2.update(neighbours)
                                    neighbours2.update(self.get_adjacent(i2,j2))
                                    if (np.array(neighbours2.values()) % 2 == 0).sum() == 2:
                                        for key in neighbours2:
                                            self.set_tent(*key)
        return self
                    
    def _is_plausible(self):
        unknowns = self.grid == 0
        tents = self.grid == 2
        
        if (self.top.sum() != self.left.sum() or
            np.any(tents.sum(axis = 0) > self.top) or np.any(tents.sum(axis = 1) > self.left) or 
            np.any((tents+unknowns).sum(axis = 0) < self.top) or
            np.any((tents+unknowns).sum(axis = 1) < self.left)):
            return False
        return True
    
    def _are_trees_plausible(self):
        unknowns = self.grid != 1
        tents = np.zeros(self.grid.shape)
        
        if (self.top.sum() != self.left.sum() or
            np.any(tents.sum(axis = 0) > self.top) or np.any(tents.sum(axis = 1) > self.left) or 
            np.any((tents+unknowns).sum(axis = 0) < self.top) or
            np.any((tents+unknowns).sum(axis = 1) < self.left)):
            return False
        return True   
    
    def reset(self):
        self.grid[self.grid != 1] = 0
        self.log = []
    
    def solve(self, deductions = "all", guesses = 0):        
#         if guesses == 0:
#             print unicode(self)
        """back-tracking algorithm
        deductions can be all, none, adjacent, or numbers"""
    
        if deductions == "all":
            apply_deductions = lambda x: x._not_adjacent_to_tree()._row_col_deductions()._pairing_deductions()
        elif deductions == "none":
            apply_deductions = lambda x: x
        elif deductions == "adjacent":
            apply_deductions = lambda x: x._not_adjacent_to_tree()
        elif deductions == "numbers":
            apply_deductions = lambda x: x._row_col_deductions()
        else:
            raise NotImplementedError('available options are: "all", "none", "adjacent" and "numbers"')
        if not self._is_plausible():
            return False
        num_unknowns = (self.grid == 0).sum()
        if num_unknowns == 0:
            return self.is_correct()
        num_unknowns_old = -1
        #recursively apply deductions
        self._not_adjacent_to_tree()
        while num_unknowns != num_unknowns_old:
            num_unknowns_old = num_unknowns
            self._row_col_deductions()
            num_unknowns = (self.grid == 0).sum()        
        num_unknowns_old = -1
        while num_unknowns != num_unknowns_old:
            num_unknowns_old = num_unknowns
#             print num_unknowns
            self._pairing_deductions()
            num_unknowns = (self.grid == 0).sum()
        
        if num_unknowns == 0:
            return self.is_correct()

#         print " "*(5*guesses),num_unknowns
        #need to make a guess
        #random for fun
        loglen = len(self.log)
#         coordinates = zip(*map(list,np.where(self.grid == 0)))
#         i,j = random.choice(coordinates)

        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i,j] == 0:
                    break
            if self.grid[i,j] == 0:
                break
        
        self.set_tent(i,j)
        if self.solve(deductions=deductions, guesses = guesses + 1):
            return True
        while len(self.log) > loglen:
            self.back()
        self.set_blank(i,j)
        return self.solve(deductions=deductions, guesses = guesses)
            
    def __unicode__(self):
        d = {0:' ',
             1:u'T',
             2:u'\U0000028C',
             3:'_'}
        s = u" "
        s += u"".join(map(str, self.top)) + "\n"
        for i in range(len(self.left)):
            s += str(self.left[i])
            s += u"".join(map(lambda x: d[x], self.grid[i])) + "\n"
        return s
    

In [ ]:
VERTEX_RADIUS = 40
MIN_VERT_SEGMENT = 80
MIN_HOR_SEGMENT = 80
DILATE = 5

SCALES = {
    22:0.81,
    21:0.85,
    20:0.9,
    19:0.95,
    18:1.00,
    17:1.06,
    16:1.12,
    15:1.19,
    14:1.27,
    13:1.36,
    10:1.73,
    8:2.14,
    7:2.38,
    6:2.75,
    5:3.21
}


screen = take_screenshot()[500:]
# show_image(screen)
canny = cv2.Canny(screen, 100, 200)
vertical_lines = np.mean(canny, axis = 0)
horizontal_lines = np.mean(canny, axis = 1)
vertical_lines /= vertical_lines.sum()
horizontal_lines /= horizontal_lines.sum()
gaussian_filter = gaussian(100, 4)
gaussian_filter /= np.linalg.norm(gaussian_filter)
smoothed_vertical = np.convolve(vertical_lines, gaussian_filter, mode = "same")
smoothed_horizontal = np.convolve(horizontal_lines, gaussian_filter, mode = "same")
# plt.plot(vertical_lines)
# plt.plot(horizontal_lines)



# canny_dilated = convolve(canny, np.ones((2 * DILATE - 1, 2 * DILATE - 1)), mode = "same")
# canny_dilated = canny_dilated >= 1
# show_image(canny_dilated, 9, 16)

kernel = np.ones((2 * DILATE - 1, 2 * DILATE - 1))
# canny_dilate = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
canny_dilate = cv2.dilate(canny, kernel)
show_image(canny_dilate)
hor_kernel = np.ones((1,MIN_HOR_SEGMENT))
vert_kernel = np.ones((MIN_VERT_SEGMENT, 1))
hor = cv2.morphologyEx(canny_dilate, cv2.MORPH_OPEN, hor_kernel)
vert = cv2.morphologyEx(canny_dilate, cv2.MORPH_OPEN, vert_kernel)
show_image(hor)
show_image(vert)

hor_lines = hor.mean(axis = 1)
vert_lines = vert.mean(axis = 0)

plt.plot(hor_lines)
plt.show()
plt.plot(vert_lines)
plt.show()

def get_peaks(lines, threshold = 0.7, peak_radius = VERTEX_RADIUS):
    peaks = []
    chopped_up = lines.copy()
    m = chopped_up.max()
    chopped_up[chopped_up < threshold * m] = 0
    while chopped_up.max() > 0:
        maxi = np.argmax(chopped_up)
        bin_left = max(maxi - peak_radius,0)
        bin_right = min(maxi + peak_radius, chopped_up.size)
        peak = chopped_up[bin_left: bin_right]
        peak /= peak.sum() # normalize to compute weighted mean of peak
        peak = int((peak * np.arange(peak.size)).sum()) # normalized mean
        peaks.append(peak + bin_left)  
        chopped_up[bin_left: bin_right] = 0
    return np.array(sorted(zip(peaks)))


hor_peaks = get_peaks(hor_lines)
vert_peaks = get_peaks(vert_lines)

print hor_peaks.reshape(-1)
print vert_peaks.reshape(-1)


# plt.plot(vert_peaks)
# plt.plot(fit_vert_peaks)
# plt.show()
# plt.plot(hor_peaks)
# plt.plot(fit_hor_peaks)
# plt.show()

gm = GridModel()
message = gm.fit(hor_peaks, vert_peaks)
if gm.success:
    fit_hor_peaks, fit_vert_peaks = gm.transform()
else:
    print "error", message
print gm.n, gm.hor_offset, gm.vert_offset, gm.sep
game_size = gm.n - 1
if game_size in SCALES:
    scale = SCALES[game_size]
else:
    lower_bound = SCALES[min([key for key in SCALES if key > game_size])]
    upper_bound = SCALES[max([key for key in SCALES if key < game_size])]
    scale, _ = find_template_scale(Image("screen", screen), TREE, scales = np.linspace(lower_bound, upper_bound))
print "scale", scale
top = []
left = []
for i in range(game_size):
    top.append( which_digit(screen[int(gm.hor_offset - gm.sep) : int(gm.hor_offset),
                                   int(gm.vert_offset  + gm.sep * i) : int(gm.vert_offset + gm.sep * (i + 1))],
                            scale = scale)[0])
    left.append(which_digit(screen[int(gm.hor_offset + gm.sep * i) : int(gm.hor_offset + gm.sep * (i + 1)),
                                   int(gm.vert_offset  - gm.sep * 0.8) : int(gm.vert_offset)],
                            scale = scale)[0])
    
print top
print left


fig = plt.figure()
fig.set_figheight(16)
fig.set_figwidth(9)
ax = fig.add_subplot(111)
ax.imshow(screen[:,:,(2,1,0)])
# ax.vlines(vert_peaks, hor_peaks.min(), hor_peaks.max(), colors = "red")
# ax.hlines(hor_peaks, vert_peaks.min(), vert_peaks.max(), colors = "red")
ax.vlines(fit_vert_peaks, fit_hor_peaks.min() - gm.sep , fit_hor_peaks.max(), colors = "blue")
ax.hlines(fit_hor_peaks, fit_vert_peaks.min() - gm.sep, fit_vert_peaks.max(), colors = "blue")
fig.show()


game_array = np.zeros((game_size, game_size))
for i in range(game_size):
    for j in range(game_size):
        image = screen[int(gm.hor_offset  + gm.sep * i) : int(gm.hor_offset  + gm.sep * (i + 1)),
                       int(gm.vert_offset + gm.sep * j) : int(gm.vert_offset + gm.sep * (j + 1))]
        canny = cv2.Canny(image, 100, 200)
        _, _, confidence = search_ar(canny, TREE.scale_canny(scale))
        if confidence > THRESHOLD:
            game_array[i][j] = 1
            continue
        _, _, confidence = search_ar(canny, TENT.scale_canny(scale))
        if confidence > THRESHOLD:
            game_array[i][j] = 2
            continue
            
game = State(game_array, top, left)
print unicode(game)
print "plausible" if game._is_plausible() else "not plausible"
game.solve()
print unicode(game)
print "solved" if game.is_correct() else "failed"

In [ ]:
THRESHOLD = 0.4

class RefreshScreenshot(Thread):
    def __init__(self, screenshot, sleep_time = 0):
        super(RefreshScreenshot, self).__init__()
        self.RUN = True
        self.screenshot = screenshot
        self.sleep_time = sleep_time
        
    def run(self):
        print "{}: screen capture thread started".format(now())
        while self.RUN:
            #timestamp of when the screenshot was requested. Screenshot is guarenteed to be newer than t.
            t = datetime.now()
            screen = take_screenshot()
            if t > self.screenshot.time: 
                self.screenshot.time = t
                self.screenshot.im = screen
                self.screenshot._canny = None
                self.screenshot._scale_canny = {}
                self.screenshot._scale_gray = {}
                self.screenshot._gray = None
            sleep(self.sleep_time)
        print "{}: screen capture thread ended".format(now())
    
    def go(self):
        self.RUN = True
    def kill(self):
        self.RUN = False

class ReadScreenshot(Thread):
    def __init__(self, screenshot, game):
        super(ReadScreenshot, self).__init__()
        self.screenshot = screenshot
        self.RUN = True
        self.is_read = False
        self.state = None
        self.game = game
        
        
    def run(self):
        print "{}: screen parse thread started".format(now())
        VERTEX_RADIUS = 40
        MIN_VERT_SEGMENT = 80
        MIN_HOR_SEGMENT = 80
        TREE_CONFIDENCE = 0.4
        DILATE = 5
        self.Y_OFFSET = 500
        self.SCALES = {
            22:0.81,#guess
            21:0.85,#guess
            20:0.9,#guess
            19:0.95,#guess
            18:1.00,
            17:1.06,
            16:1.12,
            15:1.19,
            14:1.27,
            13:1.36,
            10:1.73,
            8:2.14,
            7:2.38,
            6:2.75,
            5:3.21
        }
        
        while self.RUN:
            screen = self.screenshot.im.copy()[self.Y_OFFSET:]
            screen_image = Image("screen", screen)
            self.screen_capture_time = self.screenshot.time
            canny = cv2.Canny(screen, 100, 200)
            kernel = np.ones((2 * DILATE - 1, 2 * DILATE - 1))
            canny_dilate = cv2.dilate(canny, kernel)
            hor_kernel = np.ones((1,MIN_HOR_SEGMENT))
            vert_kernel = np.ones((MIN_VERT_SEGMENT, 1))
            hor = cv2.morphologyEx(canny_dilate, cv2.MORPH_OPEN, hor_kernel)
            vert = cv2.morphologyEx(canny_dilate, cv2.MORPH_OPEN, vert_kernel)
            hor_lines = hor.mean(axis = 1)
            vert_lines = vert.mean(axis = 0)
            scale, confidence = find_template_scale(screen_image, TREE, scales = np.array(self.SCALES.values()))
            if confidence < TREE_CONFIDENCE:
                print "no trees found"
                self.game.clear()
                sleep(0.5)
                continue
            if self._parse_grid(hor_lines, vert_lines, VERTEX_RADIUS):
                if self._construct_state(screen_image):
                    self._update_game()
                    continue
            if self._parse_trees(screen_image, scale):
                if self._construct_state(screen_image):
                    self._update_game()
                    continue
            print "{}: parse failed".format(now())
            self.game.clear()
            sleep(1)
        print "{}: screen parse thread ended".format(now())
        return  
        
    def go(self):
        self.RUN = True
    def kill(self):
        self.RUN = False
        
    def _parse_grid(self, hor_lines, vert_lines, VERTEX_RADIUS):
        for tolerance in [0.1,0.2,0.3]:
            for threshold in [0.9, 0.8, 0.7, 0.6, 0.5]:
                hor_peaks = non_maximum_suppression(hor_lines, threshold, peak_radius= VERTEX_RADIUS)
                vert_peaks = non_maximum_suppression(vert_lines, threshold, peak_radius= VERTEX_RADIUS)
                self.gm = GridModel()
                fit_hor_peaks, fit_vert_peaks = self.gm.fit_transform(hor_peaks, vert_peaks, tolerance = tolerance)
                if self.gm.success:
                    return True
        return False
    
    def _parse_trees(self, screen_image, scale):
        print "failed to find grid, trying to infer grid from trees"
        # try to determine the grid based on locations of trees and digits
        trees = match_canny_scale(screen_image, TREE, threshold=0.4, scale = scale)

        digits = []
        for key in DIGITS:
            digit = match_gray_scale(screen_image, DIGITS[key], threshold=0.9, scale = scale)
            digits += digit
        template_x, template_y = map(combine_similar, zip(*(trees + digits)))
        print "x", template_x
        print "y", template_y
        self.gm = GridModel()
        print "error:", self.gm.fit(template_y, template_x, tolerance = 0.2, symmetric = False)
        if self.gm.success:
            best_score = np.inf
            best_size = -1
            for size, actual_scale in self.SCALES.items():
                score = abs(scale - actual_scale)
                if score < best_score:
                    best_score = score
                    best_size = size

#                     gm.n = best_size + 1
            self.gm.hor_offset += self.gm.sep/2
            self.gm.vert_offset += self.gm.sep/2
            return True
        return False
    
    def _construct_state(self, screen_image):
        screen = screen_image.im
        self.sep = self.gm.sep
        self.hor_offset = self.gm.hor_offset
        self.vert_offset = self.gm.vert_offset
        self.game_size = self.gm.n - 1
        if self.game_size in self.SCALES:
            scale = self.SCALES[self.game_size]
        else:
            lower_bound = self.SCALES[min([key for key in self.SCALES if key > self.game_size])]
            upper_bound = self.SCALES[max([key for key in self.SCALES if key < self.game_size])]
            scale, _ = find_template_scale(screen_image,
                                           TREE,
                                           scales = np.linspace(lower_bound, upper_bound))
        top = []
        left = []
        for i in range(self.game_size):
            top.append( which_digit(
                screen[int(self.hor_offset - self.sep) : int(self.hor_offset),
                       int(self.vert_offset  + self.sep * i) : int(self.vert_offset + self.sep * (i + 1))],
                       scale = scale)[0])
            left.append(which_digit(
                screen[int(self.hor_offset + self.sep * i) : int(self.hor_offset + self.sep * (i + 1)),
                       int(self.vert_offset  - self.sep * 0.8) : int(self.vert_offset)],
                       scale = scale)[0])

        game_array = np.zeros((self.game_size, self.game_size))
        for i in range(self.game_size):
            for j in range(self.game_size):
                image = screen[int(self.hor_offset  + self.sep * i) : int(self.hor_offset  + self.sep * (i + 1)),
                               int(self.vert_offset + self.sep * j) : int(self.vert_offset + self.sep * (j + 1))]
                canny = cv2.Canny(image, 100, 200)
                _, _, confidence = search_ar(canny, TREE.scale_canny(scale))
                if confidence > THRESHOLD:
                    game_array[i][j] = 1
                    continue
                _, _, confidence = search_ar(canny, TENT.scale_canny(scale))
                if confidence > THRESHOLD:
                    game_array[i][j] = 2
                    continue
                image = screen[int(self.hor_offset  + self.sep * (i + 0.25)):
                               int(self.hor_offset  + self.sep * (i + 0.75)),
                               int(self.vert_offset + self.sep * (j + 0.25)):
                               int(self.vert_offset + self.sep * (j + 0.75))]
                hueimage = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                green = cv2.inRange(hueimage, (30, 25, 25), (90, 255,255))
                if green.mean() > 100:
                    game_array[i][j] = 3
                    continue
                else:
                    game_array[i][j] = 0
                    continue

        self.state = State(game_array, top, left)
        return self.state._are_trees_plausible()
    
    def _update_game(self):
        self.game.update({"state": self.state,
                          "separation": self.sep,
                          "y offset": self.hor_offset + self.Y_OFFSET,
                          "x offset": self.vert_offset,
                          "size": self.game_size,
                          "screen capture time": self.screen_capture_time})
        self.is_read = True
        print "{}: screen parsed".format(now())
        return
            
        
        
class PressButtons(Thread):
    def __init__(self, screenshot):
        super(PressButtons, self).__init__()
        self.RUN = True
        self.screenshot = screenshot
        
    def run(self):
        print "{}: continue thread started".format(now())
        while self.RUN:
            self._search_and_tap(NEXT_PUZZLE_BUTTON)
            self._search_and_tap(OK_BUTTON)
            sleep(1)
        print "{}: continue thread ended".format(now())
        
    def _search_and_tap(self, template):
        x, y, confidence = search_canny(self.screenshot, template)
        if confidence > template.tolerance:
            tap(x,y)
            print "{}: {:25} found with confidence {:3.0f}%, tapping {:4d}-{:4d}".format(
                now(), template.name, confidence * 100., x, y)
            sleep(1)
            return True
        else:
            return False
        
    def go(self):
        self.RUN = True
    def kill(self):
        self.RUN = False
                    
class PlayGame(Thread):
    def __init__(self, game, tap_time = 0.25):
        super(PlayGame, self).__init__()
        self.RUN = True
        self.is_read = False
        self.game = game
        self.tap_time = tap_time
        
    def run(self):
        print "{}: solve thread started".format(now())
        TAP_FREQUENCY = 3
        while self.RUN:
            game = self.game.get("state", False)
            if not game:
                sleep(self.tap_time)
                continue
            try:
                self.sim
            except:
                self.sim = State(np.zeros((5,5)), np.zeros(5), np.zeros(5))            
                last_tap = np.ones(game.grid.shape) * (time.time() - 5*TAP_FREQUENCY)
                
            if not np.array_equal(game.grid == 1, self.sim.grid == 1):
                print "{}: new game found".format(now())
                last_tap = np.ones(game.grid.shape) * (time.time() - 5*TAP_FREQUENCY)
                self.sim = State(game.grid, game.top, game.left)
                self.sim.reset()
                print "{}: solving game".format(now())
                self.sim.solve()
                print "{}: game solved".format(now())
                
            if np.array_equal(game.grid, self.sim.grid):
                sleep(self.tap_time)
                continue

            for i in range(game.size):
                for j in range(game.size):
                    time_since_tap = time.time() - last_tap[i,j]
                    time_since_screencap = (datetime.now()-self.game["screen capture time"]).total_seconds()
                    if (game.grid[i,j] != self.sim.grid[i,j] and
                       time_since_tap - time_since_screencap > TAP_FREQUENCY and
                       self.RUN):
                        x = int(self.game["x offset"] + self.game["separation"] * (j + 0.5))
                        y = int(self.game["y offset"] + self.game["separation"] * (i + 0.5))
                        if (game.grid[i,j] == 0 and self.sim.grid[i,j] == 2 or
                            game.grid[i,j] == 2 and self.sim.grid[i,j] == 3 or
                            game.grid[i,j] == 3 and self.sim.grid[i,j] == 0):
                            number_of_taps = 2
                        else:
                            number_of_taps = 1                            
                        for _ in range(number_of_taps):
                            tap(x,y)
                            sleep(self.tap_time)
                            last_tap[i,j] = time.time()
                        break
                else:
                    continue
                break
            sleep(self.tap_time)
        print "{}: solve thread ended".format(now())
    def go(self):
        self.RUN = True
    def kill(self):
        self.RUN = False
        

class Launcher(Thread):
    def __init__(self, tap_time = 0.25, screenshot_refresh_rate = 0.2):
        super(Launcher, self).__init__()
        self.RUN = True
        self.game = {}
        self.screenshot = Screenshot("screenshot", take_screenshot(), datetime.now())
        self.tap_time = tap_time
        self.screenshot_refresh_rate = screenshot_refresh_rate
        
    def run(self):
        print "{}: launcher thread started".format(now())
        self.refresh_thread = RefreshScreenshot(self.screenshot, self.screenshot_refresh_rate)
        self.continue_thread = PressButtons(self.screenshot)
        self.parse_thread = ReadScreenshot(self.screenshot, self.game)
        self.solve_thread = PlayGame(self.game, self.tap_time)
        self.refresh_thread.start()
        self.parse_thread.start()
        self.solve_thread.start()
        self.continue_thread.start()
        while self.RUN:
            if not self.refresh_thread.is_alive():
                self.refresh_thread = RefreshScreenshot(self.screenshot, self.screenshot_refresh_rate)
                self.refresh_thread.start()
            if not self.continue_thread.is_alive():
                self.continue_thread = PressButtons(self.screenshot)
                self.continue_thread.start()
            if not self.parse_thread.is_alive():
                self.parse_thread = ReadScreenshot(self.screenshot, self.game)
                self.parse_thread.start()
            if not self.refresh_thread.is_alive():
                self.solve_thread = PlayGame(self.game)
                self.solve_thread.start()
            sleep(1)
        self.refresh_thread.kill()
        self.continue_thread.kill()
        self.parse_thread.kill()
        self.solve_thread.kill()
        print "{}: launcher thread ended".format(now())
            
        
        
    def go(self):
        self.RUN = True
    def kill(self):
        self.RUN = False

In [ ]:
app.parse_thread.state.__dict__

In [14]:
app = Launcher(0.05)
app.start()

2021-02-18 22:51:58: launcher thread started
2021-02-18 22:51:58: screen capture thread started
2021-02-18 22:51:58: screen parse thread started
2021-02-18 22:51:58: solve thread started2021-02-18 22:51:58: continue thread started

2021-02-18 22:51:59: screen parsed
2021-02-18 22:51:59: new game found
2021-02-18 22:51:59: solving game
2021-02-18 22:52:00: game solved
2021-02-18 22:52:00: screen parsed
2021-02-18 22:52:02: screen parsed
2021-02-18 22:52:03: screen parsed
2021-02-18 22:52:05: screen parsed
2021-02-18 22:52:08: screen parsed
2021-02-18 22:52:11: screen parsed
2021-02-18 22:52:14: screen parsed
2021-02-18 22:52:17: screen parsed
2021-02-18 22:52:19: screen parsed
2021-02-18 22:52:22: screen parsed
2021-02-18 22:52:24: screen parsed
2021-02-18 22:52:27: screen parsed
2021-02-18 22:52:30: screen parsed
2021-02-18 22:52:32: screen parsed
2021-02-18 22:52:35: screen parsed
2021-02-18 22:52:37: screen parsed
2021-02-18 22:52:39: screen parsed
2021-02-18 22:52:41: screen parsed


In [15]:
app.kill()

2021-02-18 22:52:59: launcher thread ended2021-02-18 22:52:59: play button               found with confidence  85%, tapping  770-1286

2021-02-18 22:52:59: screen capture thread ended
2021-02-18 22:52:59: solve thread ended
no trees found
2021-02-18 22:53:01: screen parse thread ended
2021-02-18 22:53:01: continue thread ended


In [ ]:
%matplotlib notebook
from matplotlib.animation import FuncAnimation 
cmap = matplotlib.colors.ListedColormap(['black', 'darkgreen', 'red', 'palegreen'])
bounds=[-0.5, 0.5, 1.5, 2.5, 3.5]
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

fig = plt.figure()
axes = []
axes.append(plt.subplot(121))
axes.append(plt.subplot(222))
axes.append(plt.subplot(224))
for ax in axes:
    ax.axis("off")
im1 = axes[0].imshow(app.screenshot.im[:, :, (2,1,0)])
try:
    im2 = axes[1].imshow(app.game["state"].grid, cmap = cmap, norm = norm)
    im3 = axes[2].imshow(app.solve_thread.sim.grid, cmap = cmap, norm = norm)
except:
    pass

def update(i):
    try:
        im1.set_data(app.screenshot.im[:, :, (2,1,0)])
        im2.set_data(app.game["state"].grid)
        im3.set_data(app.solve_thread.sim.grid)
    except:
        im2 = axes[1].imshow(app.game["state"].grid, cmap = cmap, norm = norm)
        im3 = axes[2].imshow(app.solve_thread.sim.grid, cmap = cmap, norm = norm)


anim = FuncAnimation(fig, update, blit = True)

In [ ]:
im = cv2.namedWindow("image", cv2.WINDOW_NORMAL)
cv2.resizeWindow("image", 600, 600)
cv2.imshow('image',app.solve_thread.sim.grid)
cv2.waitKey(1)
cv2.destroyWindow("image")

In [ ]:
im = cv2.namedWindow("image", cv2.WINDOW_NORMAL)
cv2.resizeWindow("image", 600, 600)
while True:
    cv2.imshow('image',app.game["state"].grid)
    cv2.waitKey(1)

In [ ]:
cmap = matplotlib.colors.ListedColormap(['black', 'darkgreen', 'r', 'palegreen'])
bounds=[-0.5, 0.5, 1.5, 2.5, 3.5]
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
plt.imshow(app.solve_thread.sim.grid, cmap = cmap, norm = norm)
plt.show()
plt.imshow(app.game["state"].grid, cmap = cmap, norm = norm)
plt.show()

In [ ]:
print unicode(parse_thread.state)
print parse_thread.state._are_trees_plausible()

In [ ]:
fig = plt.figure()
fig.set_figheight(16)
fig.set_figwidth(9)
ax = fig.add_subplot(111)
ax.imshow(screen[:,:,(2,1,0)])
# ax.vlines(vert_peaks, hor_peaks.min(), hor_peaks.max(), colors = "red")
# ax.hlines(hor_peaks, vert_peaks.min(), vert_peaks.max(), colors = "red")
sep = parse_thread.sep
hor_offset = parse_thread.hor_offset + 500
vert_offset = parse_thread.vert_offset
for i in range(parse_thread.game_size + 1):
    ax.vlines(vert_offset + i * sep,
              hor_offset - sep,
              hor_offset + sep * parse_thread.game_size , colors = "blue")
    ax.hlines(hor_offset + i * sep,
              vert_offset - sep,
              vert_offset + sep * parse_thread.game_size , colors = "blue")

l = [(728, 851), (994, 984), (595, 1250), (861, 585), (196, 984), (196, 851), (994, 585), (196, 585), (462, 452)]
ax.scatter(*zip(*map(lambda (x,y) : (x,y+500) , l)))
l = [(592, 312), (989, 310), (66, 1240), (457, 310), (324, 310), (66, 576), (66, 842), (66, 975), (66, 1108), (861, 315), (728, 315), (196, 315), (72, 713), (71, 448)]
ax.scatter(*zip(*map(lambda (x,y) : (x,y+500) , l)))
fig.show()

In [ ]:
(datetime.now()-game["screen capture time"]).total_seconds()

In [ ]:
print unicode(game["state"])

In [ ]:
import cProfile
screenshot = Screenshot("screenshot", take_screenshot(), datetime.now())
refresh_thread = RefreshScreenshot(screenshot, 0.2)
continue_thread = PressButtons(screenshot)
game = {}
parse_thread = ReadScreenshot(screenshot, game)
solve_thread = PlayGame(game)
game = {}
cProfile.run("parse_thread.run()")
cProfile.run("solve_thread.run()")

# Gaussian Mixture Model

The model is a sum of gaussian packets and some constant noise
$$f(x) = a \sum_i^N g_i(x) + b \\
g_i(x) = \exp\left(-\tfrac{1}{2} h_i(x)^2\right)\\
h_i(x) = \frac{x-o-i\Delta}{\sigma}$$
With constants: $a$, $b$, $o$, $\Delta$, $\sigma$
Update function:
$$\theta = \theta_0 + \gamma\delta\partial_\theta f\\
\delta = (f(x) - y).$$
The gradient of $f$ is
$$\begin{eqnarray}\partial_af &=& \sum g_i\\
\partial_bf &=& 1\\
\partial_of &=& a \sum g_ih_i\\
\partial_\Delta f &=& a \sum i g_i h_i\\
\partial_\sigma f &=& a \sum g_i h_i / \sigma\end{eqnarray}$$




In [ ]:
N = 9
o = -100.
delta = 117.
sigma = 8.
c = 0.002
model = 0
data = vert_lines
def gaussian_f(x, mu = 0., sigma = 1.):
    return np.exp(-0.5((x - mu)/sigma)**2) / (sigma*np.sqrt(2 * np.pi))

def gaussian_vectorized(mu, sigma):
    return np.vectorize(lambda x: gaussian_f(x, mu, sigma))

model = 0                                              
for i in range(1,N+1):
    model += np.exp(-0.5 * ((np.arange(data.size) - o - i * delta) / sigma)**2)
model /= (N * sigma * np.sqrt(2*np.pi))
model += c
plt.plot(model/model.sum())
plt.plot(data/data.sum())

In [ ]:
data = vert_lines
data[data < data.max() *0.7] = 0
plt.plot(data)

In [ ]:
from PIL import Image
im = Image.fromarray(canny)
im.save("8x8canny.png")

In [ ]:
import numpy as np

def memoize_i(*keys):
    
    def wrapper(func):
        cache = dict()
        def memoized_func(*args, **kwargs):
            cache_key = (args[2], tuple(kwargs[key] for key in keys))
            if cache_key in cache:
                return cache[cache_key]
            result = func(*args, **kwargs)
            cache[cache_key] = result
            return result
        return memoized_func
    return wrapper

def memoize(*keys):
    def wrapper(func):
        cache = dict()
        def memoized_func(*args, **kwargs):
            cache_key = tuple(kwargs[key] for key in keys)
            if cache_key in cache:
                return cache[cache_key]
            result = func(*args, **kwargs)
            cache[cache_key] = result
            return result
        return memoized_func
    return wrapper

class GaussPacketModel(object):
    
    def __init__(self):    
        self.N_ITER = 10000
        self.PRINT_RATE = 1000
        self.LEARNING_RATE = 0.5
    
#     @staticmethod
#     @memoize_i("o", "delta", "sigma")
    def h(self,x,i,**params):
        o = params["o"]
        delta = params["delta"]
        sigma = params["sigma"]
        return (x - o - i * delta) / sigma
    
    
#     @staticmethod
#     @memoize_i("o", "delta", "sigma")
    def g(self,x,i,**params):
        return np.exp(-0.5 * self.h(x,i,**params)**2)
    
#     @staticmethod
#     @memoize("a", "b", "N", "o", "delta", "sigma")
    def f(self,x,**params):
        a = params["a"]
        b = params["b"]
        N = params["N"]
        return a * sum(self.g(x,i,**params) for i in range(1,N+1)) + b
    
#     @staticmethod
#     @memoize("N", "o", "delta", "sigma")
    def f_a(self,x,**params):
        return sum(self.g(x,i,**params) for i in range(1,N+1))
    
#     @staticmethod
#     @memoize()
    def f_b(self,x,**params):
        return 1    
             
#     @memoize("a", "N", "o", "delta", "sigma")
    def f_o(self,x,**params):
        return params["a"] * sum(self.g(x,i,**params) * self.h(x,i,**params) for i in range(1,N+1))
    
             
#     @memoize("a", "N", "o", "delta", "sigma")
    def f_delta(self,x,**params):
        return params["a"] * sum(i * self.g(x,i,**params) * self.h(x,i,**params) for i in range(1,N+1))
             
#     @memoize("a", "N", "o", "delta", "sigma")
    def f_sigma(self,x,**params):
        return params["a"] * sum(self.g(x,i,**params) * self.h(x,i,**params) / params["sigma"] for i in range(1,N+1))
    
    
    def fit(self,x,y):
        best_mse = np.inf
        best_params = {}
        for N in range(19,20):
            params = {"a" :20, "b" : 3.0, "o" : 0., "delta" : 130., "sigma" : 8., "N": N}
            for i in range(self.N_ITER):
                j = np.random.randint(0, x.size)
                x_point = np.array(x[j])
                y_point = np.array(y[j])
                delta = self.f(x_point, **params) - y_point
                update = {}
                if params["a"] < 0:
                    update["a"] = -2 * params["a"]
                else:
                    update["a"]     = - self.LEARNING_RATE * (delta * self.f_a(x_point,     **params)).mean()
                update["b"]     = - self.LEARNING_RATE * (delta * self.f_b(x_point,     **params)).mean()
                update["o"]     = - self.LEARNING_RATE * (delta * self.f_o(x_point,     **params)).mean()
                if params["delta"] < 0:
                    update["delta"] = -2 * params["delta"]
                else:
                    update["delta"] = - self.LEARNING_RATE * (delta * self.f_delta(x_point, **params)).mean()
                update["sigma"] = - self.LEARNING_RATE * (delta * self.f_sigma(x_point, **params)).mean()
                for key in update:
                    params[key] += update[key]
                delta = self.f(x, **params) - y
                mse = (delta**2).mean()
                if mse < best_mse:
                    best_mse = mse
                    best_params = params.copy()
                if i % self.PRINT_RATE == 0:
                    print N, mse, x_point, y_point,j
                self.params = best_params.copy()
                self.mse = best_mse
        return best_params, best_mse
    
    def predict(self, x):
        return self.f(x,**self.params)


In [ ]:
gpm = GaussPacketModel()
N = 19
x = np.arange(1080)
y = data/data.sum()
print gpm.fit(x, y)
plt.plot(gpm.predict(x))
plt.plot(y)

In [ ]:
plt.plot(gpm.predict(x))
# plt.plot(y)

In [ ]:
plt.plot(smoothed_vertical/smoothed_vertical.max())

In [ ]:
%matplotlib inline
show_image(take_screenshot())

In [ ]:
screen = Image("screen", take_screenshot())

In [ ]:
find_template_scale(screen, TREE)

In [ ]:
screen.show()

In [ ]:
show_image(cv2.matchTemplate(Image("", take_screenshot()).canny(), TREE.canny(), method = cv2.TM_CCORR_NORMED) > 0.4)